In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
# nc -lk 9999                      # для трансляции данных по сети


# lines = spark \                  # 
#     .readStream \                # инит для стриминга
#     .format("socket") \
#     .option("host", "localhost") \
#     .option("port", 9999) \
#     .load()
# lines.printSchema()

In [4]:
# дата статическая из файла
static = spark.read.option("header","true").csv("tweets.txt")

In [6]:
static.printSchema()

root
 |-- 2011042708200365|{"text":"@amberback lmao these are great!!! He says some great stuff.": string (nullable = true)
 |-- "in_reply_to_user_id":18350445: string (nullable = true)
 |-- "truncated":false: string (nullable = true)
 |-- "favorited":false: string (nullable = true)
 |-- "id_str":"63215819580125184": string (nullable = true)
 |-- "entities":{"user_mentions":[{"indices":[0: string (nullable = true)
 |-- 10]: string (nullable = true)
 |-- "screen_name":"amberback": string (nullable = true)
 |-- "id_str":"18350445": string (nullable = true)
 |-- "name":"Amber Back": string (nullable = true)
 |-- "id":18350445}]: string (nullable = true)
 |-- "hashtags":[]: string (nullable = true)
 |-- "urls":[]}: string (nullable = true)
 |-- "coordinates":null: string (nullable = true)
 |-- "geo":null: string (nullable = true)
 |-- "retweet_count":0: string (nullable = true)
 |-- "source":"\u003Ca href=\"http:\/\/tapbots.com\/tweetbot\" rel=\"nofollow\"\u003ETweetbot for iPhone\u003C\/a

In [7]:
static.take(1)

[Row(2011042708200365|{"text":"@amberback lmao these are great!!! He says some great stuff."='20110427082003510|{"text":"Rehab Nursing Tech - HealthSouth:  ( #Sewickley ', "in_reply_to_user_id":18350445=' PA) http:\\/\\/bit.ly\\/gUJbfn #therapyjobs  #Nursing #Jobs #Job #TweetMyJOBS"', "truncated":false='"in_reply_to_user_id":null', "favorited":false='"truncated":false', "id_str":"63215819580125184"='"favorited":false', "entities":{"user_mentions":[{"indices":[0='"id_str":"63215822486765568"', 10]='"entities":{"user_mentions":[]', "screen_name":"amberback"='"hashtags":[{"text":"Sewickley"', "id_str":"18350445"='"indices":[37', "name":"Amber Back"='47]}', "id":18350445}]='{"text":"therapyjobs"', "hashtags":[]='"indices":[75', "urls":[]}='87]}', "coordinates":null='{"text":"Nursing"', "geo":null='"indices":[89', "retweet_count":0='97]}', "source":"\u003Ca href=\"http:\/\/tapbots.com\/tweetbot\" rel=\"nofollow\"\u003ETweetbot for iPhone\u003C\/a\u003E"='{"text":"Jobs"', "place":{"country":

In [10]:
# загрузка данных в поток
    # `readStream` =  прямое чтение данных
    # `maxFilesPerTrigger`: размер потока в кол-вах строк / файлов

streaming = spark.readStream\
            .option("maxFilesPerTrigger", 10)\
            .schema(static.schema)\
            .csv("tweets.txt")

# проверка, что это не статика, а стриминг
streaming.isStreaming

True

In [11]:
streaming.printSchema()

root
 |-- 2011042708200365|{"text":"@amberback lmao these are great!!! He says some great stuff.": string (nullable = true)
 |-- "in_reply_to_user_id":18350445: string (nullable = true)
 |-- "truncated":false: string (nullable = true)
 |-- "favorited":false: string (nullable = true)
 |-- "id_str":"63215819580125184": string (nullable = true)
 |-- "entities":{"user_mentions":[{"indices":[0: string (nullable = true)
 |-- 10]: string (nullable = true)
 |-- "screen_name":"amberback": string (nullable = true)
 |-- "id_str":"18350445": string (nullable = true)
 |-- "name":"Amber Back": string (nullable = true)
 |-- "id":18350445}]: string (nullable = true)
 |-- "hashtags":[]: string (nullable = true)
 |-- "urls":[]}: string (nullable = true)
 |-- "coordinates":null: string (nullable = true)
 |-- "geo":null: string (nullable = true)
 |-- "retweet_count":0: string (nullable = true)
 |-- "source":"\u003Ca href=\"http:\/\/tapbots.com\/tweetbot\" rel=\"nofollow\"\u003ETweetbot for iPhone\u003C\/a

In [12]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col

# EXPLODE 
from pyspark.sql.functions import explode

In [14]:
# новая колонка в данных
words = streaming.withColumn("split", split(col('"id":63215819580125184'), ":")).withColumn("word", explode(col("split")))
words.printSchema()

root
 |-- 2011042708200365|{"text":"@amberback lmao these are great!!! He says some great stuff.": string (nullable = true)
 |-- "in_reply_to_user_id":18350445: string (nullable = true)
 |-- "truncated":false: string (nullable = true)
 |-- "favorited":false: string (nullable = true)
 |-- "id_str":"63215819580125184": string (nullable = true)
 |-- "entities":{"user_mentions":[{"indices":[0: string (nullable = true)
 |-- 10]: string (nullable = true)
 |-- "screen_name":"amberback": string (nullable = true)
 |-- "id_str":"18350445": string (nullable = true)
 |-- "name":"Amber Back": string (nullable = true)
 |-- "id":18350445}]: string (nullable = true)
 |-- "hashtags":[]: string (nullable = true)
 |-- "urls":[]}: string (nullable = true)
 |-- "coordinates":null: string (nullable = true)
 |-- "geo":null: string (nullable = true)
 |-- "retweet_count":0: string (nullable = true)
 |-- "source":"\u003Ca href=\"http:\/\/tapbots.com\/tweetbot\" rel=\"nofollow\"\u003ETweetbot for iPhone\u003C\/a

In [15]:
wordCounts = words.groupBy("word").count()
wordCounts.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [18]:
# Пример STREAMQUERY: 
# ex: activityQuery
    # indicate OUTPUTMODE: complete
    # indicate output FORMAT: memory (как вычислять)
    # NAME
    # START 

query = wordCounts\
    .writeStream\
    .outputMode("complete")\
    .queryName("trending")\
    .format("memory")\
    .start()

In [19]:
# ACTIVE проверка
spark.streams.active

[]